## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-12-06-52-55 +0000,nyt,"Turkish Military Plane Crashes in Georgia, Kil...",https://www.nytimes.com/2025/11/12/world/middl...
1,2025-11-12-06-51-13 +0000,nyt,"Jack Schlossberg, Kennedy Heir, to Seek Nadler...",https://www.nytimes.com/2025/11/11/nyregion/ja...
2,2025-11-12-06-47-00 +0000,wsj,LVMH Acquires Minority Stake in Swiss Watchmak...,https://www.wsj.com/business/deals/lvmh-acquir...
3,2025-11-12-06-34-16 +0000,nypost,Olympic figure skating legend Surya Bonaly rev...,https://nypost.com/2025/11/12/us-news/olympic-...
4,2025-11-12-06-27-45 +0000,nypost,"Turkish military cargo plane crashes, killing ...",https://nypost.com/2025/11/12/world-news/turke...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
63,trump,30
235,shutdown,24
53,government,19
81,china,14
50,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
116,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,93
215,2025-11-11-17-01-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,88
43,2025-11-12-02-00-00 +0000,wsj,As data went dark from the government shutdown...,https://www.wsj.com/economy/central-banking/th...,84
94,2025-11-11-23-36-03 +0000,nypost,Supreme Court extends order blocking full SNAP...,https://nypost.com/2025/11/11/us-news/supreme-...,83
212,2025-11-11-17-04-30 +0000,nypost,Bill to end government shutdown would let sena...,https://nypost.com/2025/11/11/us-news/bill-to-...,82


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
116,93,2025-11-11-22-13-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
85,76,2025-11-11-23-52-00 +0000,wsj,The Trump administration is poised to unveil a...,https://www.wsj.com/business/energy-oil/trump-...
319,48,2025-11-11-08-43-05 +0000,wsj,"Stock Market News, Nov. 11, 2025: Dow Jumps, N...",https://www.wsj.com/livecoverage/stock-market-...
167,43,2025-11-11-19-22-21 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
3,41,2025-11-12-06-34-16 +0000,nypost,Olympic figure skating legend Surya Bonaly rev...,https://nypost.com/2025/11/12/us-news/olympic-...
26,35,2025-11-12-03-32-40 +0000,nypost,Bank of America CEO Brian Moynihan seeks ‘impo...,https://nypost.com/2025/11/11/business/bank-of...
56,34,2025-11-12-00-40-54 +0000,nypost,Empire State Building finally lights up for Eu...,https://nypost.com/2025/11/11/us-news/empire-s...
168,34,2025-11-11-19-03-05 +0000,nypost,China plans to block rare earth shipments to U...,https://nypost.com/2025/11/11/business/china-p...
38,33,2025-11-12-02-36-43 +0000,nypost,"Beloved TV chef Mynie Steffens, 43, killed in ...",https://nypost.com/2025/11/11/world-news/belov...
128,32,2025-11-11-21-43-46 +0000,nypost,Barack Obama greets Honor Flight veterans in s...,https://nypost.com/2025/11/11/us-news/barack-o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
